In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Using TPU
import tensorflow as tf
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
#tpu_strategy = tf.distribute.TPUStrategy(tpu)

#Data Cleaning & Preparing:

In [3]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('/content/drive/MyDrive/PROJECTS/SEQUENCE CLASSIFICATION USING BERT/spamham.csv',encoding= 'unicode_escape')

In [5]:
df.columns

Index(['Category', 'Message'], dtype='object')

In [6]:
df['Message']=df['Message'].str.lower()

In [7]:
df['Category'].unique()

array(['ham', 'spam', '{"mode":"full"'], dtype=object)

In [8]:
df

,Category,Message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."
...,...,...
5568,ham,will ü b going to esplanade fr home?
5569,ham,"pity, * was in mood for that. so...any other s..."
5570,ham,the guy did some bitching but i acted like i'd...
5571,ham,rofl. its true to its name


In [9]:
df['Category'].value_counts()

ham               4825
spam               747
{"mode":"full"       1
Name: Category, dtype: int64

In [10]:
#encode spam ham to 1 & 0
df['spam'] = df['Category'].apply(lambda X:1 if X=='spam' else 0)

In [11]:
import string
puncts = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', puncts))

In [12]:
import re
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [13]:
df["Message"] = df["Message"].apply(lambda text: remove_punctuation(text))

In [14]:
df["Message"] = df["Message"].apply(lambda text: remove_emoji(text))

In [15]:
df

,Category,Message,spam
0,ham,go until jurong point crazy available only in ...,0
1,ham,ok lar joking wif u oni,0
2,spam,free entry in 2 a wkly comp to win fa cup fina...,1
3,ham,u dun say so early hor u c already then say,0
4,ham,nah i dont think he goes to usf he lives aroun...,0
...,...,...,...
5568,ham,will ü b going to esplanade fr home,0
5569,ham,pity was in mood for that soany other suggest...,0
5570,ham,the guy did some bitching but i acted like id ...,0
5571,ham,rofl its true to its name,0


#Install Transformers:

In [16]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 3.7 MB/s 
     |████████████████████████████████| 895 kB 43.5 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 3.3 MB 21.9 MB/s 
     |████████████████████████████████| 636 kB 45.5 MB/s 
     |████████████████████████████████| 109 kB 40.3 MB/s 
     |████████████████████████████████| 546 kB 55.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.2 MB/s 


#Using BERT Model:

In [ ]:
from transformers import BertTokenizer
from transformers import TFBertModel

In [ ]:
t=BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [19]:
from sklearn.model_selection import train_test_split

In [17]:
X = np.array(df['Message'])
y = np.array(df['spam'])

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [21]:
X_train

array(['had your contract mobile 11 mnths latest motorola nokia etc all free double mins  text on orange tariffs text yes for callback no to remove from records',
       'did u got that persons story',
       'im viveki got call from your number', ...,
       'aiyo her lesson so early im still sleepin haha okie u go home liao den confirm w me lor',
       'sorry ill call later', 'wake me up at  ltgt  am morning'],
      dtype=object)

In [ ]:
d=t(X_train.tolist(),max_length=100,padding='max_length',truncation=True)

In [ ]:
d.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
len(d['input_ids'][0])


100

In [ ]:
#Features or Bert Encodings
q=np.array(d['input_ids'])
q1=np.array(d['attention_mask'])

In [ ]:
with tpu_strategy.scope():
  model=TFBertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


#BUILD THE MODEL THROUGH LSTMS :

In [ ]:
import tensorflow

In [ ]:
from tensorflow.keras.layers import Input,Dense,GlobalAveragePooling1D,GlobalMaxPooling1D,Dropout,LSTM,Bidirectional,Concatenate
from tensorflow.keras.models import Model
with tpu_strategy.scope():
  inp1=Input(shape=(100,),dtype='int32')
  inp2=Input(shape=(100,),dtype='int32')
  emb=model(inp1,attention_mask=inp2)[0]
  print(emb)
  l=Bidirectional(LSTM(64,return_sequences=True))(emb)
  l=Bidirectional(LSTM(32,return_sequences=True))(l)
  la=GlobalMaxPooling1D()(l)
  l=GlobalAveragePooling1D()(l)
  l=Concatenate()([l,la])
  l=Dense(300,activation='relu')(l)
  l=Dense(64,activation='relu')(l)
  l=Dropout(0.5)(l)
  l=Dense(2,activation='softmax')(l)
  m=Model(inputs=[inp1,inp2],outputs=l)

KerasTensor(type_spec=TensorSpec(shape=(None, 100, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_bert_model'")


In [ ]:
model(inp1,attention_mask=inp2)[0]

<KerasTensor: shape=(None, 100, 768) dtype=float32 (created by layer 'tf_bert_model')>

In [ ]:
model.trainable=False

In [ ]:
m.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 100, 128)     426496      tf_bert_model[0][0]          

In [ ]:
d1=t(X_test.tolist(),max_length=100,padding='max_length',truncation=True)

In [ ]:
q2=np.array(d1['input_ids'])
q3=np.array(d1['attention_mask'])

#Compile:

In [ ]:
import keras
with tpu_strategy.scope():
  m.compile(optimizer=tf.keras.optimizers.Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy'],steps_per_execution=32)

#Fit the model

In [ ]:
m.fit([q,q1],y_train,epochs=10,batch_size=64)

Epoch 1/10


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 100) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 100) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None,) dtype=int64>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'while/Const:0' shape=() dtype=int32>, <tf.Tensor 'while/cond_8/Identity:0' shape=(None, 100) dtype=int64>, <tf.Tensor 'while/cond_8/Identity_1:0' shape=(None, 100) dtype=int64>, <tf.Tensor 'while/cond_8/Identity_2:0' shape=(None,) dtype=int64>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 100) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 100) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None,) dtype=int64>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'while/Const:0' shape=() dtype=int32>, <tf.Tensor 'while/cond_8/Identity:0' shape=(None, 100) dtype=int64>, <tf.Tensor 'while/cond_8/Identity_1:0' shape=(None, 100) dtype=int64>, <tf.Tensor 'while/cond_8/Identity_2:0' shape=(None,) dtype=int64>]


70/70 [==============================] - 76s 1s/step - loss: 0.1079 - accuracy: 0.9628
Epoch 2/10
70/70 [==============================] - 3s 37ms/step - loss: 0.0472 - accuracy: 0.9845
Epoch 3/10
70/70 [==============================] - 3s 37ms/step - loss: 0.0345 - accuracy: 0.9904
Epoch 4/10
70/70 [==============================] - 3s 37ms/step - loss: 0.0245 - accuracy: 0.9915
Epoch 5/10
70/70 [==============================] - 3s 37ms/step - loss: 0.0225 - accuracy: 0.9924
Epoch 6/10
70/70 [==============================] - 3s 37ms/step - loss: 0.0198 - accuracy: 0.9939
Epoch 7/10
70/70 [==============================] - 3s 37ms/step - loss: 0.0127 - accuracy: 0.9953
Epoch 8/10
70/70 [==============================] - 3s 37ms/step - loss: 0.0175 - accuracy: 0.9937
Epoch 9/10
70/70 [==============================] - 3s 37ms/step - loss: 0.0176 - accuracy: 0.9935
Epoch 10/10
70/70 [==============================] - 3s 37ms/step - loss: 0.0099 - accuracy: 0.9971


In [ ]:
res1 = m.predict([q2,q3])

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 100) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 100) dtype=int64>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'while/Const:0' shape=() dtype=int32>, <tf.Tensor 'while/cond_8/Identity:0' shape=(None, 100) dtype=int64>, <tf.Tensor 'while/cond_8/Identity_1:0' shape=(None, 100) dtype=int64>]


#Successful test data predictions:

In [ ]:
res1

array([[9.9999309e-01, 6.9350644e-06],
       [9.9992526e-01, 7.4691263e-05],
       [9.9999976e-01, 2.8382195e-07],
       ...,
       [9.9999869e-01, 1.2999319e-06],
       [9.9999988e-01, 9.2264415e-08],
       [9.9986196e-01, 1.3806377e-04]], dtype=float32)

In [ ]:
p=np.argmax(res1,axis=1)

In [ ]:
p

array([0, 0, 0, ..., 0, 0, 0])

#Test data predictions:

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,p,digits=3))

              precision    recall  f1-score   support

           0      0.993     0.997     0.995       961
           1      0.980     0.955     0.967       154

    accuracy                          0.991      1115
   macro avg      0.986     0.976     0.981      1115
weighted avg      0.991     0.991     0.991      1115



In [ ]:
cm = confusion_matrix(y_test,p)

In [ ]:
cm

array([[958,   3],
       [  7, 147]])